# Manage Qiskit Serverless compute and data resources



<details>
  <summary><b>Package versions</b></summary>

  The code on this page was developed using the following requirements.
  We recommend using these versions or newer.

  ```
  qiskit[all]~=1.3.1
  qiskit-ibm-runtime~=0.34.0
  qiskit-aer~=0.15.1
  qiskit-serverless~=0.18.0
  qiskit-ibm-catalog~=0.2
  qiskit-addon-sqd~=0.8.1
  qiskit-addon-utils~=0.1.0
  qiskit-addon-mpf~=0.2.0
  scipy~=1.14.1
  qiskit-addon-aqc-tensor~=0.1.2
  qiskit-addon-obp~=0.1.0
  scipy~=1.14.1
  pyscf~=2.7.0
  ```
</details>



Qiskit Serverless allows you to manage compute and data across your [Qiskit pattern](./intro-to-patterns), including CPUs, QPUs, and other compute accelerators.

## Parallel workflows



For classical tasks that can be parallelized, use the `@distribute_task` decorater to define compute requirements needed to perform a task. Start by recalling the `transpile_remote.py` example from the [Write your first Qiskit Serverless program](./serverless-first-program) topic:



In [2]:
%%writefile ./source_files/transpile_remote.py

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_serverless import distribute_task

# If you have not previously saved your credentials, follow instructions at
# https://docs.quantum.ibm.com/guides/setup-channel#iqp
# to authenticate with your API token.
service = QiskitRuntimeService()

@distribute_task(target={"cpu": 1})
def transpile_remote(circuit, optimization_level, backend):
    """Transpiles an abstract circuit (or list of circuits) into an ISA circuit for a given backend."""
    pass_manager = generate_preset_pass_manager(
        optimization_level=optimization_level,
        backend=service.backend(backend)
    )
    isa_circuit = pass_manager.run(circuit)
    return isa_circuit

Writing ./source_files/transpile_remote.py


In this example, you decorated the `transpile_remote()` function with `@distribute_task(target={"cpu": 1})`. When run, this creates an asynchronous parallel worker task with a single CPU core, and returns with a reference to track the worker. To fetch the result, pass the reference to the `get()` function.



In [3]:
%%writefile --append ./source_files/transpile_remote.py

from qiskit_serverless import get, get_arguments, save_result

arguments = get_arguments()
circuit = arguments.get("circuit")
optimization_level = arguments.get("optimization_level")
backend = arguments.get("backend")

transpile_worker_reference = transpile_remote(
    circuit,
    optimization_level,
    backend
)
result = get(transpile_worker_reference)
save_result(result)

Appending to ./source_files/transpile_remote.py


You can also create and run multiple parallel tasks as follows.



In [4]:
%%writefile --append ./source_files/transpile_remote.py

transpile_worker_references = [
    transpile_remote(circuit, optimization_level, backend)
    for circuit in arguments.get("circuit_list")
]

results = get(transpile_worker_references)
save_result(results)  # Overwrites any previously saved results

Appending to ./source_files/transpile_remote.py


### Explore different task configurations

You can flexibly allocate CPU, GPU, and memory for your tasks via `@distribute_task()`. For Qiskit Serverless on IBM Quantum™ Platform, each program is equipped with 16 CPU cores and 32 GB RAM, which can be allocated dynamically as needed.

CPU cores can be allocated as full CPU cores, or even fractional allocations, as shown in the following.

Memory is allocated in number of bytes. Recall that there are 1024 bytes in a kilobyte, 1024 kilobytes in a megabyte, and 1024 megabytes in a gigabyte. To allocate 2 GB of memory for your worker, you need to allocate `"mem": 2 * 1024 * 1024 * 1024`.



In [7]:
%%writefile --append ./source_files/transpile_remote.py

@distribute_task(target={
    "cpu": 16,
    "mem": 2 * 1024 * 1024 * 1024
})
def transpile_remote(circuit, optimization_level, backend):
    return None

Appending to ./source_files/transpile_remote.py


## Next steps

<Admonition type="info" title="Recommendations">
  *   See a full example that [ports existing code to Qiskit Serverless](./serverless-port-code).
  *   Read a paper in which researchers used Qiskit Serverless and quantum-centric supercomputing to [explore quantum chemistry](https://arxiv.org/abs/2405.05068v1).
</Admonition>

